<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/langdeck_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langdeck AI

In [1]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting
!pip install airtable-python-wrapper
import pandas as pd
import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import os
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#gc = gspread.authorize(GoogleCredentials.get_application_default())
from airtable import Airtable

!pip install txtai[pipeline]
!pip install sacremoses
from txtai.pipeline import Translation
translate = Translation()

!pip install -U deep-translator

!pip install gtts --upgrade
from gtts import gTTS

!export PYTHONIOENCODING=utf8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: gspread-dataframe 3.3.0
Uninstalling gspread-dataframe-3.3.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe-3.3.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe.py
Proceed (y/n)? y
  Successfully uninstalled gspread-dataframe-3.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gspread_dataframe-3.3.0-py2.py3-none-any.whl (7.9 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
Looking in indexes: https://pypi.org/sim

In [2]:
iso_codes = {"ams": "ar",
            "rus": "ru",
            "ukr": "uk",
            "rou": "ro",
            "alb": "sq",
            "eng": "en",
            "tur": "tr",
            "tig": "ti",
            "geo": "ka",
            "arm": "hy",
            "dar": "fa",
            "prs": "fa",
            "pst": "ps",
            "esp": "es",
            "pol": "pl",
            "som": "so",
            "amh": "am",
            "all": "de",
            "hun": "hu" }

In [3]:
deep_translator_vendor_codes =[
    {"ai_code":"11", "vendor":"txtai(Neuml)","pivot":"en", "f":"fetch_deep_tlr_txtai"},
    {"ai_code":"12", "vendor":"GoogleTranslator","pivot":"en", "f":"fetch_deep_tlr_google"},
    {"ai_code":"13", "vendor":"Pons","pivot":"fr", "f":"fetch_deep_tlr_pons"},   
    {"ai_code":"14", "vendor":"DeepL","pivot":"en", "f":"fetch_deep_tlr_deepl"}
]

In [4]:
def get_cc_config(f):
  vk = []
  try:
    with open(f) as vk_f:
        vk = json.load(vk_f)
  except BaseException as e:
    print(e)
  return vk

def set_cc_config (vk, f):
  # sauvegarde du fichier de paramétrage
  with open(f, 'w') as fp:
      json.dump(vk, fp)  

In [5]:
#
# Ouverture de la Sheet Centrale (corpus_central_base)
#
def get_CCDB_wb(uri):
  sheet_central = uri
  wb_central = gc.open_by_url(sheet_central)
  return wb_central

def get_CCDB_data(wb, idx):
  t_corpus = wb.get_worksheet(idx)
  data_t_corpus = t_corpus.get_all_values()
  return data_t_corpus

#
# DataFrame CC global (toutes les langues)
#
def get_ccdf_global(CCDB):
  return pd.DataFrame(CCDB[1:], columns=CCDB[0])

#
# DataFrame CC Français (invariable)
#
def get_ccdf_fr(df_cc):
  # on supprime les doublons
  return df_cc[['uid','expression','theme','état','date','commentaires','index']]

In [6]:
# Config Langues
config_languages = 'med_vac_synthese.json'
vk_languages = get_cc_config(config_languages)

In [7]:
def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, iso_code)
  except Exception as e:
    translation = ""
    pass
  return translation, 2

In [8]:
def fetch_pivot_en (txt_fr):
  try:
    translation = translate(txt_fr, "en")
    print (translation)
  except Exception as e:
    translation = ""
    pass  
  return translation

## Base Langdeck

In [9]:
db_langdeck_url = "https://docs.google.com/spreadsheets/d/1iJLt3qAKq576eNirMwcsdRltg7nerwa9N70dA0vraa8/" #db-langdeck

In [36]:
def get_df_content_trad(url):
  wb = gc.open_by_url(url)
  t_corpus = wb.worksheet("t_ec_content_trad")
  data_t_corpus = t_corpus.get_all_values()
  df_content_trad = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
  return df_content_trad

In [11]:
def is_audio(f):
  path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
  f = path + f
  result = True
  try:
    file = open(f, "r")
  except FileNotFoundError as e:
    # do nothing
    result = False

  return result

In [37]:
df_ec_trad = get_df_content_trad(db_langdeck_url)
df_ec_trad['is_audio'] = df_ec_trad['AudioTR'].apply(lambda x:is_audio(x))
df_ec_trad.loc[df_ec_trad['is_audio']==True]

,ID,EC_KeyFR,PhraseFR,PhraseTR,AudioFR,AudioTR,EC_Key,Locuteur,Sens,is_audio
0,EC03-RUS-001_01,EC03-FRA-001_01,Bonjour je suis le Médecin S. et je suis acco...,"Здравствуйте, я доктор С., и меня сопровождает...",Files_Files_/assets/audio/EC03-FRA-001_01.mp3,Files_Files_/assets/audio/EC03-RUS-001_01.mp3,EC03-RUS,Médecin,A,True
1,EC03-RUS-001_02,EC03-FRA-001_02,Comme c’est la première fois qu’on se rencontr...,"Поскольку мы впервые встретимся, я задам вам н...",Files_Files_/assets/audio/EC03-FRA-001_02.mp3,Files_Files_/assets/audio/EC03-RUS-001_02.mp3,EC03-RUS,Médecin,A,True
2,EC03-RUS-001_03,EC03-FRA-001_03,Vous êtes originaire de Géorgie ? de Russie ?,Вы из Грузии? из России?,Files_Files_/assets/audio/EC03-FRA-001_03.mp3,Files_Files_/assets/audio/EC03-RUS-001_03.mp3,EC03-RUS,Médecin,A,True
3,EC03-RUS-001_04,EC03-FRA-001_04,De Russie,Русский,Files_Files_/assets/audio/EC03-FRA-001_04.mp3,Files_Files_/assets/audio/EC03-RUS-001_04.mp3,EC03-RUS,Patient,B,True
4,EC03-RUS-001_05,EC03-FRA-001_05,Quand êtes-vous arrivée en France ?,Когда вы приехали во Францию?,Files_Files_/assets/audio/EC03-FRA-001_05.mp3,Files_Files_/assets/audio/EC03-RUS-001_05.mp3,EC03-RUS,Médecin,A,True
5,EC03-RUS-001_06,EC03-FRA-001_06,En novembre 2021,В ноябре 2021 года,Files_Files_/assets/audio/EC03-FRA-001_06.mp3,Files_Files_/assets/audio/EC03-RUS-001_06.mp3,EC03-RUS,Patient,B,True
6,EC03-RUS-001_07,EC03-FRA-001_07,Comment êtes-vous venue ?,Как ты пришел?,Files_Files_/assets/audio/EC03-FRA-001_07.mp3,Files_Files_/assets/audio/EC03-RUS-001_07.mp3,EC03-RUS,Médecin,A,True
7,EC03-RUS-001_08,EC03-FRA-001_08,"En bus, via la Grèce","На автобусе, через Грецию",Files_Files_/assets/audio/EC03-FRA-001_08.mp3,Files_Files_/assets/audio/EC03-RUS-001_08.mp3,EC03-RUS,Patient,B,True
8,EC03-RUS-001_09,EC03-FRA-001_09,Êtes-vous restés longtemps en Grèce ?,Вы долго оставались в Греции?,Files_Files_/assets/audio/EC03-FRA-001_09.mp3,Files_Files_/assets/audio/EC03-RUS-001_09.mp3,EC03-RUS,Médecin,A,True
9,EC03-RUS-001_10,EC03-FRA-001_10,12 jours.,12 дней.,Files_Files_/assets/audio/EC03-FRA-001_10.mp3,Files_Files_/assets/audio/EC03-RUS-001_10.mp3,EC03-RUS,Patient,B,True


In [38]:
df_ec_trad['language'] = df_ec_trad['EC_Key'].apply(lambda x:x[5:8].lower())

In [39]:
df_noaudio = df_ec_trad.loc[df_ec_trad['is_audio']==False]

In [33]:
def text2speech (text, language, path):
  # https://www.geeksforgeeks.org/convert-text-speech-python/
  # Passing the text and language to the engine, 
  # here we have marked slow=False. Which tells 
  # the module that the converted audio should 
  # have a high speed
  myobj = gTTS(text=text, lang=language, slow=False)
  myobj.save(path)

  return


In [32]:
iso_codes["rus"]

'ru'

In [41]:
filtre = "arm"
for key, value in df_noaudio.iterrows():
  if value["language"]==filtre:
    # on recherche la phrase à vocaliser
    phrase = value["PhraseTR"]
    print ("Traitement de " +phrase)
    path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
    text2speech (value["PhraseTR"], iso_codes[value["language"]], path+value["AudioTR"])


Traitement de Ողջույն, ես բժիշկ եմ Ս.-ն եմ, եւ ես ուղեկցվում եմ բժշկական ուսանողի կողմից:


gTTSError: ignored

In [43]:
# This module is imported so that we can 
# play the converted audio
import os
  
# The text that you want to convert to audio
mytext = 'Sunteți de acord să fiți examinat?'
  
# Language in which you want to convert
language = 'ro'
  
# Passing the text and language to the engine, 
# here we have marked slow=False. Which tells 
# the module that the converted audio should 
# have a high speed
myobj = gTTS(text=mytext, lang=language, slow=False)
f = "welcome.mp3"
path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/"
f = path + f
  
# Saving the converted audio in a mp3 file named
# welcome 
myobj.save(f)
  
# Playing the converted file
os.system("mpg321 welcome.mp3")

32512

# On cherche une traduction avec l'IA si manquante

1. On charge la base des traductions

In [22]:
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_translate_deep")
data_t_corpus = t_corpus.get_all_values()
# on recharge avec les nouvelles données
df_translate_deep =  pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
df_translate_deep = df_translate_deep[["hash_ai","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source","translation_ai"]]									

In [23]:
df_translate_deep

,hash_ai,hash,maitre,expression,language,translation,translation_pivot,translation_state,translation_ai_source,translation_ai
0,f9bc13efams12,f9bc13efams,Médicament,À quel moment de la journée vous prenez le méd...,ams,في أي وقت من اليوم تتناول الدواء؟,At what time of day do you take the medicine?,2,12,في أي وقت من اليوم تتناول الدواء؟
1,6e65dde7ams12,6e65dde7ams,Abcès,Abcès,ams,خُرّاج,Abscesses,2,12,الخراجات
2,80cbaceaams12,80cbaceaams,Abdomen,Abdomen,ams,البطن,Abdomen,2,12,البطن
3,95c4a863ams12,95c4a863ams,Fébrile,Accès fébrile concomitant chez un autre membre...,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,2,12,هل يصاحب ذلك نوبة حمى في أحد أفراد الأسرة الآخ...
4,30579682ams12,30579682ams,Accident,Accident,ams,حادث,Accident,2,12,حادثة
...,...,...,...,...,...,...,...,...,...,...
743,e352c9c7ams13,e352c9c7ams,Sinusite,Sinusite,ams,أضف تعليقًا,Sine [K],2,13,جيب الزاية
744,be431a55ams13,be431a55ams,Ulcère,Ulcère,ams,Ulcer,Ulcer,2,13,قَرْحَة <قَرْحات>
745,7019983eams13,7019983eams,Ulcère,Ulcère de l'estomac,ams,مُلْسَكَر,Stomach ulcer,2,13,قَرْحَةُ المَعِدَة
746,a2e70804ams13,a2e70804ams,Ulcère,Ulcère de l'intestin grêle,ams,أُجُزأة صغيرة من الأم,Ulcer of the small intestine,2,13,المِعَى الدَّقيق


In [ ]:
def fetch_deep_translator(hashid, language, vendor_code, row):
  iso_code = iso_codes[language]
  ai_dict = next(filter(lambda x: x.get('ai_code') == vendor_code,deep_translator_vendor_codes))
  vendor = ai_dict["vendor"]
  pivot = ai_dict["pivot"]
  if pivot == "en":
    pivot_expr = row["translation_pivot"]
  else:
    pivot_expr = row["expression"]
  message = "Appel de l'IA Deep " + vendor + " pour la langue " + iso_code + " pour l'expression " + pivot_expr


  df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
      lambda x:fetch_deep_tlr_google(x[1],x[2], filter, ai), 
      axis=1, 
      result_type="expand")

  return message

In [ ]:
hash_ai = "7c15835bukr12"
len(df_translate_deep.loc[df_translate_deep["hash_ai"]==hash_ai])

0

In [ ]:
filtre = "ams"
vc = "11"
for key, value in df_langdeck.iterrows():
  if value["language"]==filtre:
    # on recherche la clé hash+ia dana la table deep
    hash_ai = value["hash"]+vc
    found = df_translate_deep.loc[df_translate_deep["hash_ai"]==hash_ai]
    # pas trouvé, on passe la ligne pour appel ia
    if len(found)==0:
      print ("Recherche de " +hash_ai)
      print (fetch_deep_translator(value["hash"], value["language"], vc, value))


In [ ]:
def fetch_deep_tlr_txtai (input, language, filter, ai_code):

  translation = input
  state = 0 #par défaut

  if language == filter:
    iso_code = iso_codes[language]
    translation = ""
    try:
      # print (input + " : " + language)
      translation = translate(input, iso_code)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

In [ ]:

translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

In [ ]:
def fetch_deep_tlr_pons (input, language, filter, ai_code):

  translation = input
  state = 0 #par défaut

  if language == filter:
    iso_code = iso_codes[language]
    translation = ""
    try:
      # print (input + " : " + language)
      translation = PonsTranslator(source='en', target=iso_code).translate(input)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

In [ ]:
def fetch_deep_tlr_google (input, language, filter, ai_code):

  translation = input
  state = 0 #par défaut

  if language == filter:
    iso_code = iso_codes[language]
    translation = ""
    try:
      # print (input + " : " + language)
      translation = GoogleTranslator(source='auto', target=iso_code).translate(input)
      if translation != "":
        print (input + " : " + translation)
        state = 2 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code

In [ ]:
df_langdeck_ai_temp = df_langdeck.copy()
df_langdeck_ai_temp = df_langdeck_ai_temp [["hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
df_langdeck_ai_temp["translation_ai"]=""

Google Translator

In [ ]:
from deep_translator import GoogleTranslator
filter = "ams"
ai = 12
df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:fetch_deep_tlr_google(x[1],x[2], filter, ai), 
    axis=1, 
    result_type="expand")

Pons

In [ ]:
from deep_translator import PonsTranslator
filter = "ams"
ai = 13
df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:fetch_deep_tlr_pons(x[1],x[2], filter, ai), 
    axis=1, 
    result_type="expand")

In [ ]:
'''
filter = "rou"
ai = 11
# algorithme :
# si state==1(pending) ou 2(validé) => pas d'appel IA
# si state==0 => appel ia
df_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:fetch_deep_tlr_txtai(x[1],x[2], filter, ai) 
    if x[3]==0 
    else (x[0],x[3],0), 
    axis=1, 
    result_type="expand")
'''

'\nfilter = "rou"\nai = 11\n# algorithme :\n# si state==1(pending) ou 2(validé) => pas d\'appel IA\n# si state==0 => appel ia\ndf_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(\n    lambda x:fetch_translation_ia_txtai(x[1],x[2], filter, ai) \n    if x[3]==0 \n    else (x[0],x[3],0), \n    axis=1, \n    result_type="expand")\n'

In [ ]:
df_langdeck_ai_temp["hash_ai"]=df_langdeck_ai_temp[["hash","translation_ai_source"]].apply(lambda x:x[0]+str(x[1]), axis=1)

In [ ]:
df_langdeck_ai_temp = df_langdeck_ai_temp.drop(df_langdeck_ai_temp[df_langdeck_ai_temp["translation_ai_source"]==0].index)

In [ ]:
frames = [df_langdeck_ai, df_langdeck_ai_temp]
df_langdeck_ai = pd.concat(frames)

In [ ]:
df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]==13]

,hash,maitre,expression,language,translation,translation_pivot,translation_state,translation_ai_source,translation_ai,hash_ai
0,f9bc13efams,Médicament,À quel moment de la journée vous prenez le méd...,ams,في أي وقت من اليوم تتناول الدواء؟,At what time of day do you take the medicine?,2,13,أثْناءَ,f9bc13efams13
1,6e65dde7ams,Abcès,Abcès,ams,خُرّاج,Abscesses,2,13,خُرَاج <خُرَاجات>,6e65dde7ams13
2,80cbaceaams,Abdomen,Abdomen,ams,البطن,Abdomen,2,13,البَطْن,80cbaceaams13
3,95c4a863ams,Fébrile,Accès fébrile concomitant chez un autre membre...,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,0,13,,95c4a863ams13
4,30579682ams,Accident,Accident,ams,حادث,Accident,2,13,مُصَادَفَة,30579682ams13
...,...,...,...,...,...,...,...,...,...,...
5529,e352c9c7ams,Sinusite,Sinusite,ams,أضف تعليقًا,Sine [K],2,13,جيب الزاية,e352c9c7ams13
5530,be431a55ams,Ulcère,Ulcère,ams,Ulcer,Ulcer,2,13,قَرْحَة <قَرْحات>,be431a55ams13
5531,7019983eams,Ulcère,Ulcère de l'estomac,ams,مُلْسَكَر,Stomach ulcer,2,13,قَرْحَةُ المَعِدَة,7019983eams13
5532,a2e70804ams,Ulcère,Ulcère de l'intestin grêle,ams,أُجُزأة صغيرة من الأم,Ulcer of the small intestine,2,13,المِعَى الدَّقيق,a2e70804ams13


In [ ]:
try:
  t_sync_voc = wb.worksheet("t_translate_deep")
  wb.del_worksheet(t_sync_voc)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_translate_deep", 1, 1)
export_sheet = wb.worksheet("t_translate_deep")
set_with_dataframe(export_sheet, df_langdeck_ai)

Onglet inexistant !


In [ ]:

#df_langdeck["translation_ai_source"] = np.nan
df_langdeck["translation_score"] = df_langdeck["translation_state"].apply(lambda x: (1.0 if x==1 else 0.5 if x==2 else 0.0))
df_langdeck["translation_state_txt"] = df_langdeck["translation_state"].apply(lambda x: ("OK" if x==1 else "Pending" if x==2 else "Reject"))

In [ ]:
try:
  t_sync_voc = wb.worksheet("t_sync_voc")
  wb.del_worksheet(t_sync_voc)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_sync_voc", 1, 1)
export_sheet = wb.worksheet("t_sync_voc")
set_with_dataframe(export_sheet, df_langdeck)

# Vue Lexique

In [ ]:
t_lexique = wb.worksheet("t_lex_theme")
# on recharge avec les nouvelles données

### nouvelles colonnes Thèmes

In [ ]:
# on crée une liste des valeurs uniques des 3 colonnes de thèmes
frames = [df_langdeck["theme_1"],df_langdeck["theme_2"],df_langdeck["theme_3"]]
vk_theme = list(set(pd.concat(frames).to_list()))
vk_theme.remove('')

In [ ]:
# on ajoute les colonnes à partir de la liste et on valorise T/F
df_lexique = df_langdeck.copy()
df_lexique = pd.concat([df_lexique, pd.DataFrame(columns = vk_theme)])
for item in vk_theme:
  print (item)
  df_lexique[item] = df_lexique[["theme_1","theme_2","theme_3"]].apply(lambda x:True if (x[0]==item or x[1]==item or x[2]==item) else False, axis=1)

Examen du patient
Corps humain
Médecine générale
Etat civil
Hépato-Gastroentérologie
Maladie
Vaccination
Dossier du patient
Douleur


In [ ]:
frames=[]
df=pd.DataFrame()
frames.append(df)
# pour chaque thème on ajoute les expressions trouvées
for item in vk_theme:
  df_temp = df_lexique.loc[df_lexique[item] == True]
  df_temp.insert(0,'theme', item)
  df_temp.insert(0,'section', df_temp["ordre"].apply(lambda x:("Vocabulaire" if x==1 else "Expression" if x==2 else "Phrase" if x>=3 else "NONE")))
  df_temp.insert(0,'theme_hash', df_temp["hash"].apply(lambda x:"".join(item.split())+x))
  frames.append(df_temp)
  # on concatène le tout dans un new dataframe
  df_langdeck_theme = pd.concat(frames)

In [ ]:
df_langdeck_theme.iloc[0]

theme_hash                                         Examendupatient0a2cb764ams
section                                                                Phrase
theme                                                       Examen du patient
hash                                                              0a2cb764ams
uid                                                                  0a2cb764
maitre                                                                Douleur
ordre                                                                     3.0
expression                                  Où sont les douleurs exactement ?
idx                                                                         D
timestamp                                                 2022-04-01 08:25:25
language                                                                  ams
translation                                                    أين هو الألم ؟
language_fulltext                                               

In [ ]:
try:
  t_lexique = wb.worksheet("t_lex_theme")
  wb.del_worksheet(t_lexique)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_lex_theme", 1, 1)
export_sheet = wb.worksheet("t_lex_theme")
set_with_dataframe(export_sheet, df_langdeck_theme)

# FIN pour l'instant

# Appel IA Google Translate

In [ ]:
translation = translate("Forearm", "ar")
translation

'الكتاب'

In [ ]:
def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, "ar")
  except Exception as e:
    translation = ""
    pass
  return translation, 2


In [ ]:
df_txtai = df_langdeck.loc[(df_langdeck["language"]=="ams") & (df_langdeck["translation_state"]==0) & (df_langdeck["idx"]=="B")]

In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,,Arabe,...,0.0,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,,Arabe,...,0.0,True,False,False,False,False,False,False,False,False


In [ ]:
df_txtai[["translation","translation_state"]]=df_txtai[["expression","translation_state"]].apply(lambda x: txtai_alpha(x[0], "ar") if x[1]==0 else "", axis=1, result_type="expand")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,مرحباً، مرحباً، مرحباً، مرحباً، مرحباً، مرحباً...,Arabe,...,2,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,لاشتششششششششششششش,Arabe,...,2,True,False,False,False,False,False,False,False,False


## Mise à jour inutile dans Langdeck !!

In [ ]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck["last_updated"] = df_airtable_for_langdeck[["hash","unixtime"]].apply(lambda x:compare_langdeck_time(x[0],x[1]), axis=1) 
df_newly_updated_in_airtable = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["last_updated"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


### Récupération des audios

In [ ]:
df_newly_updated_in_airtable


,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext,unixtime,last_updated
78,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,ams,NaN,8745562bams,Arabe,1649684463000000000,True
187,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,ams,NaN,2bd46073ams,Arabe,1649684461000000000,True
201,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,ams,تَورُّم,80e0c323ams,Arabe,1649684462000000000,True
257,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,ams,ما سبب زيارتك؟,ba312208ams,Arabe,1649684462000000000,True
529,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,eng,Hand,2bd46073eng,Anglais,1649684461000000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5045,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,pol,Ile masz lat ?,ba312208pol,Polonais,1649684462000000000,True
5208,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,tig,NaN,8745562btig,Tigrinya,1649684463000000000,True
5317,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,tig,NaN,2bd46073tig,Tigrinya,1649684461000000000,True
5331,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,tig,ከኒና,80e0c323tig,Tigrinya,1649684462000000000,True


## Mise à jour dans Langdeck

In [ ]:
l = len(df_newly_updated_in_airtable)
# on parcoure la table cible
for i in range (2, l):
  # un enregistrement
  tuple_list = t_corpus.row_values(i)
  # on recherche la clé dans le dataframe source
  t_ = df_airtable_for_langdeck.loc[(df_airtable_for_langdeck['uid']==tuple_list[0]) & (df_airtable_for_langdeck['language']==tuple_list[6])]
  # verification du hash
  if len(t_)==1:
    t_corpus.update_cell(i, 9, t_["hash"].values[0])
  # comparaison des ts (A=AT;B=GG)
  ts_a = pd.to_datetime(t_["timestamp"].values[0], "%Y-%m-%d %H:%M:%S")
  ts_b = datetime.datetime.strptime(tuple_list[5], "%Y-%m-%d %H:%M:%S")
  # print (t_["hash"].values[0] + ": Airtable ts " + str(ts_a) + " GG ts " + str(ts_b))
  if ts_a > ts_b:
    # cas Airtable + récent => update dans Langdeck
    t_corpus.update_cell(i, 2, t_["maitre"].values[0])
    t_corpus.update_cell(i, 3, int(t_["ordre"].values[0]))
    t_corpus.update_cell(i, 4, t_["expression"].values[0])
    t_corpus.update_cell(i, 5, t_["idx"].values[0])
    if not pd.isnull(t_["translation"].values[0]):
      t_corpus.update_cell(i, 8, t_["translation"].values[0])    
  elif ts_b > ts_a:
    # cas Langdeck + récent => update translation dans AT uniqumeent !
    record = airtable.match('UID', tuple_list[0])
    if bool(record) is True:
      if not pd.isnull(tuple_list[7]):
        aDict = {}
        aDict[tuple_list[6]] = tuple_list[7]
        airtable.update(record['id'], aDict)
        print (record['id'])


    


In [ ]:
frames = [df_airtable_for_langdeck, df_langdeck]
result = pd.concat(frames, keys=["hash"])
result.drop_duplicates(subset=['hash'], inplace=True, keep='last')

In [ ]:
result

uid      maitre  ordre                                         expression idx           timestamp language                                        translation         hash language_fulltext
hash 0     f9bc13ef  Médicament      3  À quel moment de la journée vous prenez le méd...   M 2022-04-01 08:15:21      ams                  في أي وقت من اليوم تتناول الدواء؟  f9bc13efams             Arabe
     1     6e65dde7       Abcès      1                                              Abcès   A 2022-03-29 15:58:04      ams                                             خُرّاج  6e65dde7ams             Arabe
     2     80cbacea     Abdomen      1                                            Abdomen   A 2022-04-07 08:42:16      ams                                              البطن  80cbaceaams             Arabe
     3     95c4a863     Fébrile      3  Accès fébrile concomitant chez un autre membre...   F 2022-04-01 08:27:19      ams              نوبة حمى مزامنة عند فرد اخر من الأسرة  95c4a863ams             Arabe
     4     30579682    Accident      1                                           Accident   A 2022-03-29 15:58:05      ams                                               حادث  30579682ams             Arabe
...             ...         ...    ...                                                ...  ..                 ...      ...                                                ...          ...               ...
     5451  d112f379       Poids      3                        Votre poids est-il stable ?   P 2022-03-31 14:33:52      tig                                                NaN  d112f379tig          Tigrinya
     5452  2344ba84   Accoucher      3  Vous avez accouché par césarienne ou voie basse ?   A 2022-03-29 16:00:33      tig                             ብኦፕረሽን ወይ ብኖርማር ወሊድኪ ?  2344ba84tig          Tigrinya
     5453  36e66161  Contagieux      3            Y a-t-il des cas contagieux à l'école ?   C 2022-03-29 16:00:33      tig                                 አብ ትምህርቲ ዲዩ ለጊብካ ?  36e66161tig          Tigrinya
     5454  c32f5fd9   Chronique      3  Y a-t-il des maladies chroniques dans votre fa...   C 2022-04-01 08:23:46      tig                      ካብ በተሰብካ ሕዱር ሕማሕ ዘለዎ አለካ ዲዩ ?  c32f5fd9tig          Tigrinya
     5455  d3dba8e4        Zona      1                                               Zona   Z 2022-03-29 16:00:34      tig  ዞና ዝብሃል ሕማም ኣብ ቖርበት ማለት ብደገ ዝቖስል ብርቱዕ ቓንዛ ዘለዎ ...  d3dba8e4tig          Tigrinya

[5456 rows x 10 columns]

In [ ]:
df_airtable_for_langdeck[(df_airtable_for_langdeck['hash'].isin(df_langdeck['hash']) & ~df_airtable_for_langdeck['timestamp'].isin(df_langdeck['timestamp']) )]

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
2,80cbacea,Abdomen,1,Abdomen,A,2022-04-07 08:42:16,ams,البطن,80cbaceaams,Arabe


In [ ]:
set_with_dataframe(t_corpus, result)

END

### Vocabulaire avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre



In [ ]:
vk_voc = airtable.get_all(view='Vocabulary', sort=['idx_cor','Maitre','Ordre'])
df_voc = pd.DataFrame.from_records((r['fields'] for r in vk_voc))
df_voc = df_voc[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique']]

### Traductions avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre

In [ ]:
df_voc_trad = airtable.get_all(view='Traductions', sort=['idx_cor','Maitre','Ordre'])
df_voc_trad = pd.DataFrame.from_records((r['fields'] for r in df_voc_trad))
df_voc_trad = df_voc_trad[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique',
             'eng','ams','tur','rus','ukr','rou','hun','tig','alb','geo','arm',
             'dar','pst','esp','pol']]

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Lexique'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_voc_trad)

Onglet inexistant !


In [ ]:
grouped = df_voc.groupby(['idx_cor','Maitre'])
'''
for name, group in grouped:
  print (name)
  print (group["Proposition"])
'''

'\nfor name, group in grouped:\n  print (name)\n  print (group["Proposition"])\n'

Recherche des UID manquants dans Airtable

In [ ]:
def airtable_get_missing_uids(df):
  return df.loc[pd.isna(df["UID"])]


Recherche dans Airtable par Séquence

In [ ]:
def airtable_get_records_by_sequence(df, sequence):
  return df.loc[df["Séquence"]==sequence]


In [ ]:
#df_vkat.loc[df_vkat["Séquence"]=="1. Présentation + antécédents"]

In [ ]:
airtable_get_missing_uids(df_vkat)

,Proposition,CATEGRAM,DEPENDANCE,idx_cor,TYPE,UID,VOCABULAIRE,Thématique,idx,Ordre,...,Updates,Séquence,MedGen,Statut,Fiche,Phrase Assoc,Définition,Corpus copy,DOMAINE,Exp Assoc
6,~ chirurgicale,[exp],[reco8iFRctnJgMJCl],I,expression,NaN,NaN,[Vaccination],I,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Accouchement,[nm],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Accoucher,[vi],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Appareil ~,[exp],NaN,D,expression,NaN,NaN,[Médecine générale],NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Assurance,[nf],NaN,A,vocabulaire,NaN,Général,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Urine,[nf],NaN,U,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Vaccinal,[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,"Vacciné, -e",[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Votre poids a-t-il augmenté ?,[phrase],[recnFsFSoE1DCjYqt],P,phrase,NaN,NaN,[Médecine générale],NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rceherche des termes présents dans Central, présents dans AIRTBL mais sans UID

In [ ]:
vk_at_trad = airtable.get_all(view='Traductions',sort='Proposition')
df_vk_at_trad = pd.DataFrame.from_records((r['fields'] for r in vk_at_trad))
df_vk_at_trad = df_vk_at_trad[["Proposition","idx_cor","UID"]]
df_vk_at_trad_notna = df_vk_at_trad.loc[pd.notna(df_vk_at_trad["UID"])]
df_vk_at_trad_na = df_vk_at_trad.loc[pd.isna(df_vk_at_trad["UID"])]

### merge côté AT : avec les UID présents 

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_notna, how="right", left_on='uid', right_on='UID')

## On complète avec les traductions

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    aDict = {}
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)


## On complète avec les UID manquants

In [ ]:
df_vk_at_trad_na

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_na, how="right", left_on='expression', right_on='Proposition')

In [ ]:
df_merged = df_merged.loc[pd.notna(df_merged["uid"])]

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('Proposition', row["expression"])
    aDict = {}
    key = "UID"
    aDict[key] = row["uid"]
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)

## On vide toute la base AT dans Central Consolidation

In [ ]:
vk_at = airtable.get_all(view='Admin',sort='Proposition')
df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Termes'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_vkat)



END

In [ ]:
record = airtable.match('UID', '30579682')
fields = {'eng': 'Accident'}
airtable.update(record['id'], fields)

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'eng': 'Accident',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
record

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
def airtable_table_get(uri, params, headers):
  airtable_records = []
  run = True
  while run is True:
    response = requests.get(uri, params=params, headers=headers)
    airtable_response = response.json()
    airtable_records += (airtable_response['records'])
    if 'offset' in airtable_response:
      run = True
      params = (('offset', airtable_response['offset']),)
    else:
      run = False
  return airtable_records

# Table Corpus

In [ ]:

url = "https://api.airtable.com/v0/" + base_id + "/" + table_name


### On lit tous les enregistrements, puis on met dans un dataframe

In [ ]:
airtable_records = airtable_table_get(url, params, headers)

airtable_rows = [] 
airtable_index = []
for record in airtable_records:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df = pd.DataFrame(airtable_rows, index=airtable_index)
df = df [['Proposition','UID', 'Name ID', 'DEPENDANCE', 'idx', 'idx_cor','Statut','Thématique']]
df.sort_values(by=['Proposition'], ascending=True, inplace=True)

In [ ]:
df.head(10)

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recKbPHqDTXNPwYK0,A jeun,f397cda1,recKbPHqDTXNPwYK0,NaN,J,J,NaN,Vaccination
rec1qTnfWxjhx6SBl,AME - Aide médicale d'État,f49563a9,rec1qTnfWxjhx6SBl,NaN,A,A,NaN,Vaccination
rec0HgtD22F7TWM3W,Abcès,6e65dde7,rec0HgtD22F7TWM3W,NaN,A,A,NaN,Médecine générale
recGaRQGjV5qN3QBN,Accident,30579682,recGaRQGjV5qN3QBN,NaN,A,A,NaN,Vaccination
rec3MxDq67MC982ng,Accident cérébral,6f214e4c,rec3MxDq67MC982ng,[recGaRQGjV5qN3QBN],A,A,NaN,Vaccination
reciexfGbNKvXzQg8,Alcool,NaN,reciexfGbNKvXzQg8,NaN,A,A,NaN,Médecine générale
recfY4uKkgK8Jp8GO,Allergie,8b195cf4,recfY4uKkgK8Jp8GO,NaN,A,A,NaN,Vaccination
recSKSrXVLhDfMq6w,Allergie alimentaire,f754eebf,recSKSrXVLhDfMq6w,[recfY4uKkgK8Jp8GO],A,A,NaN,Vaccination
rectthHdPchXi1zr9,Allergie médicamenteuse,7256c169,rectthHdPchXi1zr9,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination
recHNzXfvWWm0Lwp8,Allergie à un médicament,316fa7ab,recHNzXfvWWm0Lwp8,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination


In [ ]:
def lookup_uid (uid):
  try:
    record = df_cc_fr.loc[df_cc_fr["uid"]==uid]["index"].item()
  except Exception as e:
    list_no_uid.append(uid)
    return uid
  if record is not None:
    return record

In [ ]:
x = df_cc_fr.loc[df_cc_fr["uid"]=="72f0e8db"]["index"].item()

In [ ]:
list_no_uid = [] # uid non présents côté central = mots à créer
df['UID'] = df['UID'].apply(lambda x: make_unique_id() if pd.isna(x) else x)
df.idx_cor = df['UID'].apply(lambda x: lookup_uid(x))

convert the dataframe to a dictionary of dictionaries.

In [ ]:
df.tail()

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recia1L6nICYg6LJ8,Zona,d3dba8e4,recia1L6nICYg6LJ8,NaN,NaN,Z,NaN,Vaccination
recVvIHdNWf80zvsG,À quelle moment de la journée vous prenez le m...,0f86d1ef,recVvIHdNWf80zvsG,[recOivQOmeragygVr],M,0f86d1ef,Ramona,Médecine générale
recWp0rajm2qKcJuF,Êtes vous vacciné (e) ?,c5df267a,recWp0rajm2qKcJuF,NaN,NaN,Ê,NaN,Médecine générale
recBY5NXGM8UtBcaX,Êtes-vous enceinte ?,7768882c,recBY5NXGM8UtBcaX,NaN,NaN,Ê,NaN,Médecine générale
reczy8Os5Hc3qZ1yA,Œdème,72f0e8db,reczy8Os5Hc3qZ1yA,NaN,NaN,Œ,NaN,Médecine générale


In [ ]:
dict_records = df.tail().to_dict(orient="index")

In [ ]:
for record_id, upload_data in dict_records.items():
  record_url = url + "/" + record_id
  upload_dict = {"records" : [{"fields" : upload_data}]}
  upload_json = json.dumps(upload_dict)
  requests.post(url, data=upload_json, headers=headers)

In [ ]:
headers

{'Authorization': 'Bearer keyrI98TIqu6mbFcf',
 'Content-Type': 'application/json'}

In [ ]:
requests.patch(url, data=upload_json, headers=headers)

<Response [422]>

In [ ]:
upload_json='{"records": [{"fields": {"Proposition": "OEDEME", "id": "reczy8Os5Hc3qZ1yA"}}]}'

In [ ]:
requests.post(record_url, data=upload_json, headers=headers)

<Response [404]>

In [ ]:
def fmt_table(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)

  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # format de l'auto wrap
  fmt = cellFormat(
      wrapStrategy='WRAP'
      )
  format_cell_range(ws, 'B:C', fmt)

  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=1)
  set_column_width(ws, 'A:C', 500)


In [ ]:
nom_onglet = 'Liste'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df)
fmt_table(wb,nom_onglet)


In [ ]:
table_name = 'Structure'
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name

In [ ]:
df

,Name,Situation de départ,Notes
recnI80hvydv71fs4,0.Conception,"Plus on possède, plus on est soi-même possédé....","Trouver le plaisir d’écrire. Partir d’idées, d..."
recI2pgkNq2iP52rm,1.Situation initiale,Un jour d'été où il pleut et fait un temps abo...,Une vengeance d'une femme éconduite lors d'un ...
recb2DqhB9GvfI12n,2.Personnage,Un homme (marié ?) qui a donné rendez-vous à u...,Un Dom Juan sûr de son charme et imbu de sa pe...
recNLdU01JpCnvknJ,3.Acte déclencheur,La femme lui propose un jeu. Elle est dans le ...,NaN
recRxZVhpSPD6oQXx,4.Réaction,L'homme est d'abord tenté de partir car il est...,NaN
recRr06fOM4hGGaLY,5.Péripéties,"Pendant la visite (ennuyeuse), l'homme tente d...",NaN
recvWnAYn9VNySVlB,6.Chute,Un spectacle où une partie du public est appel...,NaN
recDmZtMMNmiy7R0M,Récit,NaN,NaN
